## 7 - Smart Pointers
##### **Author: Adam Gatt**

### Problem: How do we know when to delete a dynamic object?

We can create objects in dynamic memory (i.e. on _the heap_) using the keywords `new` and `delete`. Objects created in this way do not follow the rules of scope like static objects do. They will not have their destructor called and memory freed when their originating scope block ends. Instead we must be sure to manually call `delete` to free the object when it is no longer needed.
* If we fail to `delete` the object its memory will remain allocated but unused, resulting in a memory leak
* If we use the object after it has been deleted we cause a [use after free](https://encyclopedia.kaspersky.com/glossary/use-after-free/) error, or potential vulnerability
* If our control logic [`delete`s the object twice](https://isocpp.org/wiki/faq/freestore-mgmt#double-delete-disaster) then we enter _undefined_ behaviour and a runtime crash or worse may occur.

We conclude that each object with a `new` statement must be matched with a corresponding `delete`. But in practice this rule may be surprisingly difficult to enforce due to complexities in reasoning about _object lifetimes_.

* The object's creation and end-of-life points might be very far away from each other in the code, requiring careful tracking of the object to see where it goes.
* The object might flow through different control logic, resulting in multiple different areas where its end-of-life is expected and `delete` must be called. The logic must be checked to ensure that only one such call can be reached for each object.
* We need to always be certain whether a variable is an array or not as we would need to call `delete[]` instead, and calling the wrong type of delete operator results in undefined behaviour
* A dynamic object might be created (and returned) by a function, showing no `new` syntax in the immediate area around where the object is first used.
* The object reference may be provided to a function call, which may "consume" the object (and thus delete it) or might not, depending on convention and how the function is expected to be used.
* The object may be provided for storage in a data structure which may itself be deleted. Some data structures may delete their held references upon destruction (e.g. a list) and some may not (e.g. a cache).
* There may be multiple references to the same dynamic object, potentially held in different threads, requiring us to be careful that only one reference is deleted and at a safe time. This often requires us to choose one reference as being responsible for object deletion.

### Object ownership

Many libraries and idioms attempt to straighten out this complexity by modelling the concept of _object ownership_. A dynamic object only ever has a single _owning reference_, with other temporary non-owning references allowed. The ownership of the object can be transferred between references, or across function boundaries, or between threads, but there is only one owner at any point in time. When the object is no longer required, the owning reference is responsible for deleting it. Non-owning references have a way to check whether the object has been deleted before attempting to make use of it.

### Smart pointers: A RAII approach to object ownership

Smart pointers are an application of the RAII technique towards memory management. We use RAII objects called _smart pointers_ to represent and track the ownership of the dynamic object. The smart pointers are then responsible for automatically deleting the object at the right time, according to the strategy used. C++ offers three types of smart pointers:

* `std::unique_ptr`

`unique_ptr` is a single unambiguous owner of the dynamic object. By using _move semantics_ (discussed more in the next notebook) we can allow for the object to be transferred to a new owner at important boundaries (e.g. function boundaries). We follow scope-based rules to ensure that the object is eventually deleted when its current owner falls out of scope.

* `std::shared_ptr`

`shared_ptr` is for situations where multiple owners is required and unavoidable. In this model, you can have multiple `shared_ptr`s to the same dynamic object, and it will be deleted when all of its `shared_ptrs` have been destroyed (i.e. at the moment when the last surviving `shared_ptr` is destroyed). This is 

* `std::weak_ptr` is a weak, non-owning reference to a shared_ptr


#### Note: Please ignore the existence of std::auto_ptr

### Intro - RAII for memory management

#### Benefits
Goal: A set of well-defined processes and reason-able rules by which we can ensure that memory leaks cannot occur.

Ideal: Just like malloc/free, never write new/delete again

#### Note: avoid auto_ptr
Are unique_ptr and shared_ptr thread-safe?

### Unique_ptr
#### Creation (Manual/make_unique)
#### Dereference operators (* and ->)
#### Transfer ownership (move semantics)
#### Returning from function

### Shared_ptr
Reference counted
#### Creation (Manual/make_shared)
#### Create extra reference
#### Drop reference
#### weak_ptr
Non-owning reference - check for null and expect to lose at any time
    Breaks cycles of shared_ptrs
Example of object cache


### Rules for smart pointers
* create smart points with their std::make_* functions
* return smart pointers by value
* Accept unique_ptr as parameter if you intend to claim ownership
* Accept shared_ptr as parameter if you intend to add to ownership
* Accept raw pointer if you intend to use but not claim ownership

### Use as data members in classes
Use in member initialisation list
Tie to object life-cycle
#### Standard pattern